In [49]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader
import ast

In [100]:
df_train = pd.read_csv('/scratch/umni5/a/shives/Course_Projects/CS_587_DL/Sets_of_Seqs/data/train.csv')
df_valid = pd.read_csv('/scratch/umni5/a/shives/Course_Projects/CS_587_DL/Sets_of_Seqs/data/valid.csv')
df_test = pd.read_csv('/scratch/umni5/a/shives/Course_Projects/CS_587_DL/Sets_of_Seqs/data/test.csv')

In [3]:
df_train

,customer_id,products_before,order_after,post_cutoff,num_products_before,num_products_post
0,597942,['Skinny Ankle R.W Brooklyn'],0,[],1,0
1,959484,"['Despacito', 'OP T-shirt Milano', 'Livia top'...",0,[],7,0
2,1324380,"['VonDuck PREMIUM', 'Barza', 'Joey', 'Bama', '...",0,[],13,0
3,1144704,['Seinfeld Long Padded'],0,[],1,0
4,669824,['Jerry tee'],0,[],1,0
...,...,...,...,...,...,...
108977,1359436,"['Granny High Rise Brief', 'Love Affair Triang...",1,"['ESSENTIAL FIONA LACE TANK', 'Maria HW Woven ...",21,10
108978,984586,"['Ludwig', 'Tahiti shirt dress', 'Alex Trs (J)']",1,['Julia RW Skinny Denim TRS'],3,1
108979,458089,[],1,"['Mom HW Shorts', 'Thyme top', 'Birdy HW']",0,3
108980,1106403,"['Puma sequin blazer V5', 'Janni', 'Sky HW jog...",1,"['Vanessa', 'Hudson PU wide leg']",6,2


BINARY CLASSIFIER

In [101]:
df_train = df_train[['products_before','order_after']]
df_valid = df_valid[['products_before','order_after']]
df_test = df_test[['products_before','order_after']]

In [102]:
df_train

,products_before,order_after
0,['Skinny Ankle R.W Brooklyn'],0
1,"['Despacito', 'OP T-shirt Milano', 'Livia top'...",0
2,"['VonDuck PREMIUM', 'Barza', 'Joey', 'Bama', '...",0
3,['Seinfeld Long Padded'],0
4,['Jerry tee'],0
...,...,...
108977,"['Granny High Rise Brief', 'Love Affair Triang...",1
108978,"['Ludwig', 'Tahiti shirt dress', 'Alex Trs (J)']",1
108979,[],1
108980,"['Puma sequin blazer V5', 'Janni', 'Sky HW jog...",1


In [103]:
print(type(df_train['products_before'][0]))

<class 'str'>


In [83]:
def product_splitting(s):
    items = ast.literal_eval(s)
    return [str(item).strip() for item in items]
    

In [84]:
df_train['products_before'] = df_train['products_before'].apply(product_splitting)
df_valid['products_before'] = df_valid['products_before'].apply(product_splitting)
df_test['products_before'] = df_test['products_before'].apply(product_splitting)

In [85]:
df_train

,products_before,order_after
0,[Skinny Ankle R.W Brooklyn],0
1,"[Despacito, OP T-shirt Milano, Livia top, carl...",0
2,"[VonDuck PREMIUM, Barza, Joey, Bama, Lee (1), ...",0
3,[Seinfeld Long Padded],0
4,[Jerry tee],0
...,...,...
108977,"[Granny High Rise Brief, Love Affair Triangle ...",1
108978,"[Ludwig, Tahiti shirt dress, Alex Trs (J)]",1
108979,[],1
108980,"[Puma sequin blazer V5, Janni, Sky HW jogger, ...",1


In [90]:
encoder_model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')

def encode_products(product_list):
    embeddings = encoder_model.encode(
        product_list,
        convert_to_numpy=True,    # get numpy array
        batch_size=32,            # adjust if you like
        show_progress_bar=False
    )
    return [embeddings[i] for i in range(embeddings.shape[0])]

In [99]:
# 1. Grab the list from row 0
sample_products = df_train.at[13, 'products_before']
print("Products at row 0:", sample_products)

# 2. Encode just that one list
sample_embeddings = encode_products(sample_products)

# 3. Inspect
print(f"Got {len(sample_embeddings)} embeddings, each of shape {sample_embeddings[0].shape}")
# e.g.:
# Got 3 embeddings, each of shape (384,)

# 4. (Optional) look at the numeric values
print(sample_embeddings)


Products at row 0: ['KARIN PADDED bra', 'SUPREME TIGHTS', 'EDC J CARLOTTA POLO TOP', 'NORWAY fleece jacket (1)', 'Queen Sweater', 'KARIN PADDED bra', 'EVA TIGHT CONSCIOUS TANK TOP', 'MILA CONSCIOUS TANK TOP', 'INGRID BASIC HOOD', 'Maria Woven Shorts', 'Olga Tank TVP', 'Havana dress SPEED', 'Havana dress SPEED', 'Goji dress w', 'Juan', 'Katie', 'Tanya mockneck LS']
Got 17 embeddings, each of shape (384,)
[array([ 1.05251446e-02,  5.92367873e-02, -5.82547579e-03,  4.97369468e-02,
       -1.05085876e-02,  9.12031438e-03,  1.25958487e-01,  5.75644448e-02,
        3.28417704e-03, -1.38263507e-02,  5.22243492e-02, -1.55548658e-02,
       -7.58927129e-03,  3.47649008e-02, -3.21174636e-02, -4.05595824e-02,
        3.32946815e-02,  1.62085388e-02, -3.17987017e-02,  5.11404239e-02,
       -1.48374576e-03, -3.93743850e-02,  5.35548516e-02,  2.32533533e-02,
        4.83887456e-02,  1.89789664e-02,  5.15772812e-02, -6.73607886e-02,
        4.51052561e-02, -5.08344248e-02, -1.02587737e-01, -3.082898

In [ ]:
df_train['products_before'] = df_train['products_before'].apply(encode_products)

In [2]:
import pandas as pd
import numpy as np
import torch
import ast
from sentence_transformers import SentenceTransformer


#Data Loading
df_train = pd.read_csv('/scratch/umni5/a/shives/Course_Projects/CS_587_DL/Sets_of_Seqs/data/train.csv')
df_valid = pd.read_csv('/scratch/umni5/a/shives/Course_Projects/CS_587_DL/Sets_of_Seqs/data/valid.csv')
df_test = pd.read_csv('/scratch/umni5/a/shives/Course_Projects/CS_587_DL/Sets_of_Seqs/data/test.csv')


#Converts the string representation of list to actual list
def product_splitting(s):
    items = ast.literal_eval(s)
    return [str(item).strip() for item in items]

for df in (df_train, df_valid, df_test):
    #df[:] = df[['products_before','post_cutoff']].copy()
    df['products_before'] = df['products_before'].apply(product_splitting)
    df['post_cutoff'] = df['post_cutoff'].apply(product_splitting)

In [6]:
all_products = set()
for df in (df_train, df_valid, df_test):
    all_products.update(df['products_before'].explode().unique())

In [8]:
len(all_products)

33353

In [9]:
model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
product_list = list(all_products)
embeddings = model.encode(
    product_list,
    convert_to_numpy=True,
    batch_size=64
)
product_embedding_dict = dict(zip(product_list, embeddings))

In [11]:
len(product_embedding_dict)

33353

In [ ]:



#Lookup function that just pulls from the dict
def encode_products_from_dict(product_list, embedding_dict=product_embedding_dict):
    # stack into a (len(product_list), D) array
    return np.vstack([embedding_dict[prod] for prod in product_list])

#Apply lookup to each row
for df in (df_train, df_valid, df_test):
    df['products_before_embeddings'] = df['products_before'].apply(encode_products_from_dict)
    df['post_cutoff_embeddings'] = df['post_cutoff'].apply(encode_products_from_dict)

# now df['{products_before, post_cutoff}_embeddings'] is an array of shape (N_items, embed_dim) per row,

